In [ ]:
import yfinance as yf
import pandas as pd


# Define the ticker symbol
ticker_symbol = "AAPL"

# Get the Ticker object
company = yf.Ticker(ticker_symbol)


# Get various types of fundamental data
info = company.info
print("Company Info:")
for key, value in info.items():
    #print(f"{key}: {value}")
    print(None)
    break


In [ ]:

import json

def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return []
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return []

def extract_tickers_and_details(companies_data):
    tickers = []
    companies_details = {}
    
    for k,company in companies_data.items():
        ticker = company['ticker']
        tickers.append(ticker)
        companies_details[ticker] = company
    
    return tickers, companies_details

def main():
    file_path = 'company.json'
    companies_data = load_json_file(file_path)
    
    if companies_data:
        tickers, companies_details = extract_tickers_and_details(companies_data)
        #print(companies_details)
        df = pd.DataFrame()
        df["Ticker"] = tickers
        df["Company"] = [c["title"] for k,c in companies_details.items()]
        print(df)
        df.to_csv("company_tickers.csv")
        #print("Company Tickers:")
        #print(len(tickers),tickers)

if __name__ == "__main__":
    main()

In [ ]:

# Get historical market data
historical_data = company.history(period="5y")
print("\nHistorical Market Data:")
print(historical_data.head())
#historical_data.to_csv("price_history.csv")
historical_data


In [ ]:
# Get financial statements
financials = company.financials
print("\nFinancial Statements:")
print(financials)

financials.to_csv("financials.csv")


In [ ]:
balance_sheet = company.balance_sheet
print("\nBalance Sheet:")
print(pd.DataFrame(balance_sheet))
balance_sheet.to_csv("balancesheet.csv")


In [ ]:
cashflow = company.cashflow
print("\nCash Flow:")
print(cashflow)



In [ ]:
# Get key metrics
key_metrics = company.major_holders
print("\nMajor Holders:")
print(key_metrics)



In [ ]:
key_metrics = company.institutional_holders
print("\nInstitutional Holders:")
print(key_metrics)

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

def get_quarter_year(date):
    q = (date.month - 1) // 3 + 1
    y = date.year
    return str(y) + "Q" + str(q)
    
# Load the list of tickers from a CSV file (assuming you have downloaded it)
#tickers = pd.read_csv('company_tickers_subset.csv', header=None, names=['Ticker'])
tickers = pd.read_csv('company_tickers_subset.csv')

def get_fundamental_data(ticker):
    try:
        company = yf.Ticker(ticker)
        info = company.info
        info = pd.DataFrame(info)
        info["ticker"] = ticker

        historical_data = company.history(period="5y")
        historical_data = pd.DataFrame(historical_data)
        historical_data["ticker"] = ticker
        historical_data.reset_index(inplace=True, drop=False)
        historical_data = historical_data.T[~historical_data.T.index.duplicated(keep='first')].T
        
        financials = company.financials
        financials = pd.DataFrame(financials)
        financials.columns = [get_quarter_year(x) for x in financials.columns]
        financials["ticker"] = ticker
        financials.reset_index(inplace=True, drop=False)
        financials = financials.T[~financials.T.index.duplicated(keep='first')].T
        
        #balance_sheet = company.balance_sheet
        balance_sheet = company.quarterly_balance_sheet
        balance_sheet = pd.DataFrame(balance_sheet)
        balance_sheet.columns = [get_quarter_year(x) for x in balance_sheet.columns]
        balance_sheet["ticker"] = ticker
        balance_sheet.reset_index(inplace=True, drop=False)
        balance_sheet = balance_sheet.T[~balance_sheet.T.index.duplicated(keep='first')].T
        
        #cash_flow = company.cashflow
        cash_flow = company.quarterly_cashflow
        cash_flow = pd.DataFrame(cash_flow)
        cash_flow.columns = [get_quarter_year(x) for x in cash_flow.columns]
        cash_flow["ticker"] = ticker
        cash_flow.reset_index(inplace=True, drop=False)
        cash_flow = cash_flow.T[~cash_flow.T.index.duplicated(keep='first')].T

        #cash_flow = company.cashflow
        is_flow = company.quarterly_income_stmt
        is_flow = pd.DataFrame(is_flow)
        is_flow.columns = [get_quarter_year(x) for x in is_flow.columns]
        is_flow["ticker"] = ticker
        is_flow.reset_index(inplace=True, drop=False)
        is_flow = is_flow.T[~is_flow.T.index.duplicated(keep='first')].T

        # Print or save the data as needed
        """
        print(f"Ticker: {ticker}")
        print("Info:")
        print(info)
        print("\nFinancials:")
        print(financials)
        print("\nBalance Sheet:")
        print(balance_sheet)
        print("\nCash Flow:")
        print(cash_flow)
        print("\n")
        """
        return historical_data, info, financials, balance_sheet, cash_flow, is_flow

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        e = pd.DataFrame()
        return e,e,e,e,e,e

def main():
    info_df = pd.DataFrame()
    hs_df = pd.DataFrame()
    financials_df = pd.DataFrame()
    balance_sheet_df = pd.DataFrame()
    cash_flow_df = pd.DataFrame()
    is_flow_df = pd.DataFrame()
    hs_df,info_df,financials_df,balance_sheet_df,cash_flow_df,is_flow_df = get_fundamental_data("KR")
    for ticker in tickers['Ticker']:    
        hs,i,f,b,c,is_f = get_fundamental_data(ticker)
        
        hs_df = pd.concat([hs,hs_df],ignore_index=True)
        info_df = pd.concat([i,info_df],ignore_index=True)
        financials_df = pd.concat([f,financials_df],ignore_index=True)
        balance_sheet_df = pd.concat([b,balance_sheet_df],ignore_index=True)
        cash_flow_df = pd.concat([c,cash_flow_df],ignore_index=True)
        is_flow_df = pd.concat([is_f,is_flow_df])

        print("success: ",ticker)
        
        # Add a delay to avoid rate limiting
        time.sleep(1)

    info_df.to_csv("company_info_2.csv")
    financials_df.to_csv("Q_financials_info_2.csv")
    balance_sheet_df.to_csv("Q_bs_info_2.csv")
    cash_flow_df.to_csv("Q_cash_flow_info_2.csv")
    #is_flow_df.to_csv("Q_is_info.csv")
    hs_df.to_csv("price_history_2.csv")

if __name__ == "__main__":
    import time
    main()

success:  KR
success:  IR
success:  FAST
success:  GIS
success:  CTVA


/var/folders/bd/4k3_gjxs037f4v_ts19mcbp00000gn/T/ipykernel_1519/1664695690.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  info_df = pd.concat([i,info_df],ignore_index=True)


success:  MSTR
success:  VRT
success:  EXC


/var/folders/bd/4k3_gjxs037f4v_ts19mcbp00000gn/T/ipykernel_1519/1664695690.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  info_df = pd.concat([i,info_df],ignore_index=True)


success:  IT


...

success:  VIV
success:  NTNX
success:  APTV


/var/folders/bd/4k3_gjxs037f4v_ts19mcbp00000gn/T/ipykernel_1519/1664695690.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  info_df = pd.concat([i,info_df],ignore_index=True)


success:  EG


/var/folders/bd/4k3_gjxs037f4v_ts19mcbp00000gn/T/ipykernel_1519/1664695690.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  info_df = pd.concat([i,info_df],ignore_index=True)


success:  WLMIY
success:  SUI
success:  AMCR


In [ ]:
info_df = pd.DataFrame()
hs_df = pd.DataFrame()
financials_df = pd.DataFrame()
balance_sheet_df = pd.DataFrame()
cash_flow_df = pd.DataFrame()
is_flow_df = pd.DataFrame()
hs,i,f,b,c,is_f = get_fundamental_data("AAPL")

hs_df = pd.concat([hs_df,hs],ignore_index=True)
info_df = pd.concat([info_df,i],ignore_index=True)
financials_df = pd.concat([financials_df,f],ignore_index=True)
balance_sheet_df = pd.concat([balance_sheet_df,b],ignore_index=True)
cash_flow_df = pd.concat([cash_flow_df,c],ignore_index=True)


In [ ]:
hs,i,f,b,c,is_f = get_fundamental_data("KR")

hs_df = pd.concat([hs_df,hs],ignore_index=True)
info_df = pd.concat([info_df,i],ignore_index=True)
financials_df = pd.concat([financials_df,f],ignore_index=True)
balance_sheet_df = pd.concat([balance_sheet_df,b],ignore_index=True)
#cash_flow_df = pd.concat([cash_flow_df,c],ignore_index=True)

In [ ]:
b.index = np.array(b.index) + 1*90

In [ ]:
np.array(b.index) + 1*90

In [ ]:
b